In [4]:
import pandas as pd
import re

In [5]:
def clean_data_long(filename: str, spaces: bool):
    df = pd.read_csv(filename, skipinitialspace=spaces)
    females = "F"
    if spaces:
        females += " "
    df = df[df["Sex"] == females] # remove male cattles
    df["Number of Cattle"] = df["Number of Cattle"].str.replace(',', '').astype(int) # convert strings to ints
    old_group_num = df.groupby("Age Groups").sum()["Number of Cattle"]

    age_groups = old_group_num.index.tolist()
    # change order of months names
    to_remove = "6 to 12 months"
    if spaces:
        to_remove += " "
    age_groups.remove(to_remove)
    age_groups[1] = to_remove
    last_group = "102+months"
    if spaces:
        last_group += " "
    age_groups.append(last_group)
    
    # unite months groups to year groups
    year_nums = []
    for i in range(0, len(age_groups), 2):
        year_nums.append(old_group_num.loc[age_groups[i]] + old_group_num.loc[age_groups[i+1]])

    return pd.Series(year_nums)

data_2015 = clean_data_long("data/great-britain-cattle-population-england-on_1-january-2015.csv", True)
data_2014 = clean_data_long("data/great-britain-cattle-population-on_1-january-2014.csv", True)
data_2013 = clean_data_long("data/gb-cattle-population-on-01-january-2013.csv", False)


In [42]:
def clean_data_short(filename, to_add=0):
    df = pd.read_csv(filename, skiprows=lambda x: x <= (1+to_add) or x > (21+to_add), usecols=[0,1,2])

    df["Number of Cattle"] = df["Dairy"].str.replace(',', '').astype(int) \
        + df["Non Dairy"].str.replace(',', '').astype(int)

    df = df.drop(18) # remove total for all ages

    age_groups = df["Number of Cattle"]
    year_nums = []

    for i in range(0, df.shape[0], 2):
        year_nums.append(age_groups.loc[i] + age_groups.loc[i+1])

    return pd.Series(year_nums)

data_2012 = clean_data_short("data/great-britain-cattle-population-on_1-january-2012.csv")
data_2011 = clean_data_short("data/great-britain-cattle-population-on_1-january-2010-to-1-january-2011.csv", 24)
data_2010 = clean_data_short("data/great-britain-cattle-population-on_1-january-2010-to-1-january-2011.csv")